### Before starting a new run, create a new qual to give to all workers

(Note that this is different from the *participation* qual, which is assigned to everyone across all runs)

(Also, in case you need it, the link to manage the quals via the Web UI: https://requester.mturk.com/qualification_types )

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import mtglobals

In [10]:
client, mturk_env = mtglobals.gen_client()

Your account balance is 498885.58


### (a) Get info for the previous (most recent) custom qual

In [11]:
qual_info = mtglobals.get_current_qual()

In [12]:
qual_info

{'qual_name': 'Workplace_Survey_Custom14',
 'qual_id': '376OKSJ89SYRLC8IL4XY8ZX2HAYX7I',
 'last_qual_num': 1,
 'last_offer_amt': '0.95'}

### (b) Enter the info for the new qual to be created

In [22]:
new_qual_name = "Workplace_Survey_Custom17"
start_qual_num = 0
# This is left over from an old version -- we don't technically need to keep track of
# the previous wage anymore, but I'm keeping it in just for ease of debugging since
# this makes it so the log files will always record each generated wage. Here it's
# just chosen at random, it'll be re-sampled anyways when the first stage-2 HIT is launched
start_offer_amt = mtglobals.random_wage()

### (c) Create the new qual via MTurk API

In [23]:
print(f"Creating new qual {new_qual_name} with start num {start_qual_num}, start wage {start_offer_amt}")

Creating new qual Workplace_Survey_Custom17 with start num 0, start wage 0.98


In [24]:
result_name, result_id = mtglobals.create_new_qual(client, new_qual_name)

In [25]:
result_name

'Workplace_Survey_Custom17'

In [26]:
result_id

'31UNB3YYU013XOAZ9X0VP9PP3HTGTL'

### (d) And update the locally-stored info on the current qual

In [27]:
mtglobals.update_current_qual(result_name, result_id, start_qual_num, start_offer_amt)

Current qual updated to be: Workplace_Survey_Custom17,31UNB3YYU013XOAZ9X0VP9PP3HTGTL,0,0.98


### Now run 01b_PostStage1.ipynb

### (old version)

(potentially useful if API acts weird and you need to just get the id for the most-recently-created qual)

In [12]:
# Get the suffix
qual_suffix = int(qual_info['qual_name'][-2:])
int(qual_suffix)
# Increment
new_suffix = qual_suffix + 1
new_suffstr = str(new_suffix).zfill(2)
# Create the new qual name
new_qual_name = "Workplace_Survey_Custom" + new_suffstr
new_qual_name
print(new_qual_name)
# Create the actual qual via mturk api
if not mtglobals.qual_exists(client, new_qual_name):
    result_name, result_id = mtglobals.create_new_qual(client, new_qual_name)
    result_id = mtglobals.get_qual_id(client, new_qual_name)
    # And set as the new current qual
    mtglobals.update_current_qual(new_qual_name, result_id, 0, '0.50')
else:
    # If it already exists, just get the id and update the current qual
    print("Qual already exists, updating current qual.txt")
    result_id = mtglobals.get_qual_id(client, new_qual_name)
    mtglobals.update_current_qual(new_qual_name, result_id, 0, '0.50')

Workplace_Survey_Custom14
Qual already exists, updating current qual.txt
Current qual updated to be: Workplace_Survey_Custom14,376OKSJ89SYRLC8IL4XY8ZX2HAYX7I,0,0.50


### Sigh. No idea why it works sometimes and doesn't work other times, but just run this to update the currentqual.txt

In [11]:
# If it already exists, just get the id and update the current qual
print("Qual already exists, updating current qual.txt")
result_id = mtglobals.get_qual_id(client, new_qual_name)
mtglobals.update_current_qual(new_qual_name, result_id, 0, '0.50')

Qual already exists, updating current qual.txt


Exception: No matching quals found!